In [11]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

provider = "google_genai" # "google_genai" / "openai"

if provider == "google_genai":
    llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
elif provider == "openai":
    llm = init_chat_model("gpt-5-mini", model_provider="openai")

In [22]:
ai_message = llm.invoke(
    "Who's your maker? Also, tell me the name of an interesting historic moment from a random century (random seed is current time)."
)

print(ai_message.content)

I am a large language model, trained by Google.

As for an interesting historic moment from a random century (simulating a random selection based on current time):

Let's go with the **14th Century CE**.

An incredibly significant and dramatic event from that time was **The Black Death**. This devastating pandemic, primarily caused by the bacterium *Yersinia pestis*, swept across Europe, Asia, and North Africa from roughly 1347 to 1351, killing an estimated 75 to 200 million people. Its impact profoundly reshaped societies, economies, and cultures for centuries to come.


In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import OpenAIEmbeddings

if provider == "google_genai":
    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
elif provider == "openai":
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [14]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [15]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

63

In [16]:
_ = vector_store.add_documents(documents=all_splits)

In [17]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [19]:
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs